# Building A Portfolio

When building a portfolio I will take into consideration the following: 
- Expected Return
- Risk (Standard Deviation)
- Sharp Ratio
- Correlation

for the purpose of this research we are using equal weight for the whole portfolio, we have 10 stock so 10% of each stock makes the entire portfolio


In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import ffn
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

In [ ]:
#Getting 10 stocks for the Portfolio
start_date = '2019-06-03'
end_date = '2024-06-04'

stocks = ['AMZN', 'CCJ', 'LMT', 'PLTR', 'PANW', 'LULU', 'SOFI', 'XOM', 'AMGN', 'AMAT'] #Portfolio 
data = yf.download(stocks, start=start_date, end=end_date)['Adj Close'] #list of Adj Closing prices

In [ ]:
data

In [ ]:
data.info()

In [ ]:
data.aggregate([min, np.mean, np.median, np.std, max,]) #Statistics of the stock over the last 5 years

In [ ]:
current_price = data.iloc[-1] #last price of the stock

In [ ]:
current_price

In [ ]:
for stock in data.columns:
    price_difference = data[stock].max() - current_price[stock]
    print(f'{stock} {price_difference}') #we can see how the stocks are priced right now compared to their max for the last 5 years. 

## Expected Return
* The profit and loss that an investor anticipates on an investment that has known historical rates of return (ROR), the expected return is usually based on historical data and is therefore not guaranteed into the future, however it does often set reasonable expectations. 

Calculating expected return
- CAMP model
- Analyst Forecast
- 

In [ ]:
#Normalize prices of the stocks
(data / data.iloc[0]*100).plot(figsize=(10,8))

In [ ]:
returns = np.log(data / data.shift(1)) * 100 #returns of the stocks (Day pct changes) 

#for SOFI has IPO June 1st 2021, and PLTR has IPO September 30th 2020, we can see the difference in the returns and we don't have data for the first two years

#to calculate SOFI Returns wi will use the first day of trading so June 2nd 2021 June 2 2021 = data.iloc[504]
sofi_returns = ((data['SOFI'].iloc[-1] - data['SOFI'].iloc[504])/data['SOFI'].iloc[504])*100

#to calculate PLTR Returns we will use the first day of trading so September 30th 2020 = data.iloc[336]
pltr_returns = ((data['PLTR'].iloc[-1] - data['PLTR'].iloc[336]) / data['PLTR'].iloc[336])*100

In [ ]:
sofi_returns

In [ ]:
pltr_returns

In [ ]:
returns.tail()

In [ ]:
#five year return include all stocks 
five_year_return = ((current_price - data.iloc[0]) / data.iloc[0]) * 100

In [ ]:
five_year_return = pd.DataFrame(five_year_return)
five_year_return.columns = ['Returns']
five_year_return.loc['PLTR'] = pltr_returns
five_year_return.loc['SOFI'] = sofi_returns
five_year_return

In [ ]:
five_year_return

In [ ]:
#Visualize 5 year returns
x = five_year_return['Returns']
y = five_year_return.index
colors = plt.get_cmap('tab10')
plt.figure(figsize=(10,6))
plt.bar(y, x, color=colors(np.arange(len(five_year_return))))
plt.ylabel('% Return')
plt.xlabel('Ticker')
plt.title('5 Year Returns')
plt.show()

In [ ]:
year1_returns = ((data.iloc[252]-data.iloc[0])/data.iloc[0])*100

In [ ]:
year2_returns = ((data.iloc[504] - data.iloc[253])/data.iloc[253])*100

In [ ]:
year3_returns = ((data.iloc[757] - data.iloc[505])/data.iloc[505])*100

In [ ]:
year4_returns = ((data.iloc[1008] - data.iloc[758])/data.iloc[758])*100

In [ ]:
year5_returns = ((data.iloc[1259] - data.iloc[1009])/data.iloc[1009])*100

In [ ]:
returns_df = pd.DataFrame([year1_returns, year2_returns, year3_returns, year4_returns, year5_returns])

returns_df.index = ['year 1', 'year 2', 'year 3', 'year 4', 'year 5']

In [ ]:
returns_df

In [ ]:
returns_df.plot(figsize=(10,10), kind='bar', legend=True, title='Yearly Returns')

In [ ]:
data

In [ ]:
d = data[['AMAT', 'AMGN', 'AMZN', 'CCJ', 'LMT', 'LULU', 'PANW', 'XOM']]
PLTR = data.PLTR
SOFI = data.SOFI 

In [ ]:
stats = d.calc_stats()
stats.display()

In [ ]:
SOFI_stats = SOFI.calc_stats()
SOFI_stats.display()

In [ ]:
PLTR_stats = PLTR.calc_stats()
PLTR_stats.display()

In [ ]:
weights = np.array([0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10])

In [ ]:
#mean daily returns and covariance of daily returns
mean_daily_returns = returns.mean()
cov_matrix = returns.cov()

In [ ]:
pf_return = np.sum(mean_daily_returns * weights) * 252
pr_return = round(pf_return, 3)

In [ ]:
pf_std_dev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(252)
pf_std_dev = round(pf_std_dev,3)

In [ ]:
print('Expected annualized return: ' + '{:.1%}'.format(pf_return))
print("Volatility: " + "{:.1%}".format(pf_std_dev))

In [ ]:
#calculate expected returns and sample covariance 
expected_returns = expected_returns.mean_historical_return(data)

In [ ]:
expected_returns

In [ ]:
risk_model = risk_models.sample_cov(data)

In [ ]:
risk_model

In [ ]:
#Optimize for maximal sharp ratio
ef = EfficientFrontier(expected_returns, risk_model)
raw_weight = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

In [ ]:
current_price = data.iloc[-1] #last price of the stock

In [ ]:
#cretate dictionary for the stock weights
w_data = {'AMZN': 0.10, 'CCJ': 0.10, 'LMT': 0.10, 'PLTR': 0.10, 'PANW': 0.10, 'LULU': 0.10, 'SOFI': 0.10, 'XOM': 0.10, 'AMGN': 0.10, 'AMAT': 0.10}

Using the Efficient Frontier for portfolio optimization can be limiting, as it tends to select the five stocks with the highest returns, making the portfolio less risk-averse

In [ ]:
da = DiscreteAllocation(w_data, current_price, total_portfolio_value=10000)
allocation, leftover = da.greedy_portfolio()
print('Discrete allocation:', allocation)
print('Funds remaining: ${:.2f}'.format(leftover))